在這個模型中，我們將3個LSTM層疊在一起，使模型能夠學習更高層次的時間表示。
前兩個LSTM返回完整的輸出序列，但最後一個只返回輸出序列的最後一步，從而降低了時間維度（即將輸入序列轉換成單個向量）。


In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# 期望输入数据尺寸: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # 返回维度为 32 的向量序列
model.add(LSTM(32, return_sequences=True))  # 返回维度为 32 的向量序列
model.add(LSTM(32))  # 返回维度为 32 的单个向量
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# 生成虚拟训练数据
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# 生成虚拟验证数据
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))



Using TensorFlow backend.


Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 2s 2ms/step - loss: 11.4657 - acc: 0.0810 - val_loss: 11.5045 - val_acc: 0.1400
Epoch 2/5
1000/1000 [==============================] - 0s 157us/step - loss: 11.4641 - acc: 0.1070 - val_loss: 11.5077 - val_acc: 0.1000
Epoch 3/5
1000/1000 [==============================] - 0s 159us/step - loss: 11.4641 - acc: 0.1070 - val_loss: 11.5109 - val_acc: 0.1400
Epoch 4/5
1000/1000 [==============================] - 0s 147us/step - loss: 11.4637 - acc: 0.1120 - val_loss: 11.5085 - val_acc: 0.1300
Epoch 5/5
1000/1000 [==============================] - 0s 172us/step - loss: 11.4630 - acc: 0.1010 - val_loss: 11.5093 - val_acc: 0.1100


帶有狀態（stateful）的相同的棧式LSTM模型
有狀態的循環神經網絡模型中，在一個批次的樣本處理完成後，其內部狀態（記憶）會被記錄並作為下一個批次的樣本的初始狀態。這允許處理更長的序列，同時保持計算複雜度的可控性。


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10
batch_size = 32

# 期望输入数据尺寸: (batch_size, timesteps, data_dim)
# 请注意，我们必须提供完整的 batch_input_shape，因为网络是有状态的。
# 第 k 批数据的第 i 个样本是第 k-1 批数据的第 i 个样本的后续。
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# 生成虚拟训练数据
x_train = np.random.random((batch_size * 10, timesteps, data_dim))
y_train = np.random.random((batch_size * 10, num_classes))

# 生成虚拟验证数据
x_val = np.random.random((batch_size * 3, timesteps, data_dim))
y_val = np.random.random((batch_size * 3, num_classes))

model.fit(x_train, y_train,
          batch_size=batch_size, epochs=5, shuffle=False,
          validation_data=(x_val, y_val))

Train on 320 samples, validate on 96 samples
Epoch 1/5
320/320 [==============================] - 2s 6ms/step - loss: 11.5547 - acc: 0.0656 - val_loss: 11.4900 - val_acc: 0.0833
Epoch 2/5
320/320 [==============================] - 0s 229us/step - loss: 11.5497 - acc: 0.0719 - val_loss: 11.4905 - val_acc: 0.0625
Epoch 3/5
320/320 [==============================] - 0s 207us/step - loss: 11.5488 - acc: 0.1031 - val_loss: 11.4907 - val_acc: 0.0521
Epoch 4/5
320/320 [==============================] - 0s 195us/step - loss: 11.5480 - acc: 0.1031 - val_loss: 11.4909 - val_acc: 0.0521
Epoch 5/5
320/320 [==============================] - 0s 189us/step - loss: 11.5472 - acc: 0.1125 - val_loss: 11.4911 - val_acc: 0.0625
